## Setup

In [ ]:
from specific import *

In [ ]:
a = ESA_CCI_Landcover_PFT()
cube = Datasets(a).select_variables("pftCrop").cube

In [ ]:
cube.name()

In [ ]:
cube_plotting(cube.data)

In [ ]:
from datetime import datetime

from wildfires.data import prepare_selection

a2 = prepare_selection(
    Datasets(a),
    min_time=datetime(2010, 1, 1),
    max_time=datetime(2015, 4, 1),
    which="climatology",
)

In [ ]:
cube_plotting(a2.cube.data)

In [ ]:
cube_plotting(cube.data)

In [ ]:
cube_plotting(cube.data.data)

In [ ]:
cube.data.min()

In [ ]:
cube.shape

In [ ]:
b = a.copy(deep=True)

In [ ]:
b.regrid()

In [ ]:
b.cube

In [ ]:
cube_plotting(b.cube.data)

In [ ]:
interp_cubes = b.interpolate_yearly_data(datetime(2010, 1, 1), datetime(2015, 4, 1))

In [ ]:
cube_plotting(interp_cubes[0].data.data > 1)

In [ ]:
cube_plotting(interp_cubes[0].data.data < 0)

In [ ]:
cube_plotting(np.mean(interp_cubes[0].data, axis=0))

In [ ]:
cube_plotting(interp_cubes[0].data.data)

In [ ]:
cube_plotting(b.cube.data.mask)

### Get unshifted data

In [ ]:
# XXX:
# data_memory.clear()

(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_data()

In [ ]:
_, _, _, filled1, _, _ = data_processing(
    Datasets([ESA_CCI_Landcover_PFT(), GFEDv4()]).select_variables(
        ("pftCrop", "GFED4 BA")
    )
)

In [ ]:
cube_plotting(filled1.select_variables("pftCrop", inplace=False).cube)

In [ ]:
selection_datasets = [
    AvitabileThurnerAGB(),
    Copernicus_SWI(),
    ERA5_Temperature(),
    ESA_CCI_Landcover_PFT(),
    GFEDv4(),
    HYDE(),
    WWLLN(),
]
# These datasets will potentially be shifted.
datasets_to_shift = [
    ERA5_DryDayPeriod(),
    MOD15A2H_LAI_fPAR(),
    VODCA(),
    GlobFluo_SIF(),
]
selection_datasets += datasets_to_shift
dataset_times(selection_datasets, lat_lon=True)[2]

In [ ]:
masked_datasets.select_variables("pftCrop", inplace=False)

In [ ]:
cube_plotting(master_mask.astype("float"))

In [ ]:
cube_plotting(
    masked_datasets.select_variables("popd", inplace=False).cube.data,
    log=True,
    title="popd",
)

In [ ]:
plt.hist(
    get_unmasked(masked_datasets.select_variables("popd", inplace=False).cube.data),
    bins=100,
)
plt.yscale("log")

In [ ]:
mask_variables = [
    "AGB Tree",
    "SWI(1)",
    "pftCrop",
    "pftHerb",
    "SIF",
    "popd",
    "FAPAR",
    "LAI",
    "VOD Ku-band",
]
for var in mask_variables:
    selected = masked_datasets.select_variables(var, inplace=False)
    cube_plotting(selected.cube.data.mask, title=f"{var} Mask")
    cube_plotting(selected.cube.data, title=f"{var} Data")

In [ ]:
cube_plotting(GFEDv4().cube.data, log=True)

In [ ]:
for cube in masked_datasets.cubes[42:]:
    print(cube.name())

In [ ]:
master_mask2 = None
cube = VODCA().cubes[0]
print(cube.shape[0])

In [ ]:
climatology = VODCA().get_climatology_dataset(
    datetime(2010, 1, 1), datetime(2015, 4, 1)
)

In [ ]:
for i in range(12):
    cube_plotting(climatology.cubes[0].data[i], title=i)

In [ ]:
for i in range(12):
    cube_plotting(climatology.cubes[0].data.mask[i], title=i)

In [ ]:
for i in range(1):
    mask = cube[314 - i].data.mask
    if master_mask2 is None:
        master_mask2 = mask.copy()
    else:
        master_mask2 |= mask
cube_plotting(master_mask2)

In [ ]:
master_mask2 = None
for cube in masked_datasets.cubes[42:50]:
    mask = cube.data.mask
    if master_mask2 is None:
        master_mask2 = mask.copy()
    else:
        master_mask2 |= mask
cube_plotting(master_mask2)

In [ ]:
plotted = set()
for cube in masked_datasets.cubes:
    if any(p in cube.name() for p in plotted):
        continue
    #     cube_plotting(cube.data.mask, title=f"{cube.name()} Mask");
    if cube.data.mask.sum() > 9593124:
        print(cube.name(), cube.data.mask.sum())
        cube_plotting(
            cube.data.mask,
            title=f"{cube.name()} Mask",
            fig=plt.figure(figsize=(25, 12)),
        )
    plotted.update((cube.name(),))

### Get shifted data

In [ ]:
(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_offset_data()

## Mapping

In [ ]:
with figure_saver("high_fapar_high_dry_day_period", sub_directory="map_plots"):
    mpl.rc("figure", figsize=(11, 4))
    constrained_map_plot(
        {"FAPAR": (0.36, None), "Dry Day Period": (18, None)},
        exog_data,
        master_mask,
        plot_variable="FAPAR",
        coastline_kwargs={"linewidth": 0.5},
    )

In [ ]:
with figure_saver("high_dry_day_period_18_medium_agbtree", sub_directory="map_plots"):
    mpl.rc("figure", figsize=(11, 4))
    constrained_map_plot(
        {"Dry Day Period -18 - -6 Month": (22, None), "AGB Tree": (0.9, 20)},
        exog_data,
        master_mask,
        plot_variable="AGB Tree",
        coastline_kwargs={"linewidth": 0.5},
    )

In [ ]:
with figure_saver("high_pftCrop", sub_directory="map_plots"):
    mpl.rc("figure", figsize=(11, 4))
    constrained_map_plot(
        {"pftCrop": (0.6, None)},
        exog_data,
        master_mask,
        plot_variable="pftCrop",
        coastline_kwargs={"linewidth": 0.5},
    )

## Correlation Plot

In [ ]:
X_corr = exog_data
with figure_saver("corr_plot"):
    corr_plot(
        shorten_columns(X_corr[X_corr.columns[:-15]]), fig_kwargs={"figsize": (12, 8)}
    )
print("Excluded columns:", shorten_columns(X_corr).columns[-15:])

In [ ]:
with figure_saver("corr_plot_full"):
    corr_plot(shorten_columns(X_corr[X_corr.columns]), fig_kwargs={"figsize": (12, 8)})